## Importação do Gurobi

In [27]:
import gurobipy as gp

## Parâmetros da Instância 1

In [28]:
# NUM_VAGOES
n = 2           
# NUM_TRECHOS
m = 6           
# NUM_ATIVIDADES
o = 15          
# NUM_TRENS
v = 2           
# Tempo de circulação de um vagão j em um trecho q
P = 5           
# Número grande o suficiente
M = 2880        
# VAGOES_TREM_SAIDA
TRAIN_OUT = [1,1]  
# VAGOES_TREM_CHEGADA
TRAIN_IN = [0,0]  
# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],
       [1,1,1,0,1,0],
       [0,1,1,1,0,0],
       [0,0,1,1,0,0],
       [0,1,0,0,1,1],
       [0,0,0,0,1,1]]   
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

## Definição de Modelo MILP

In [29]:
model = gp.Model("milp")

## Variáveis de Decisão

In [30]:
# y[i,j] = instante de início da i-ésima operação do vagão j
y = model.addVars(o,n, vtype=gp.GRB.CONTINUOUS, name="y")         
# z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
z = model.addVars(o,m,n, vtype=gp.GRB.BINARY, name="z")           
# x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
x = model.addVars(m,o,n,o,n, vtype=gp.GRB.BINARY, name="x")       
# f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
f = model.addVars(o,n, vtype=gp.GRB.BINARY, name="f")             
# c[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
c = model.addVars(n, vtype=gp.GRB.CONTINUOUS, name="c")           
# c_max = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)
c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")       

## Função Objetivo

In [31]:
model.setObjective(c_max, gp.GRB.MINIMIZE)

## Restrições do Problema

In [32]:
# Equação 2 / (4-2)
for j in range(n):
  for i in range(o-1):
    model.addConstr
    (
      y[i+1,j] 
      >= y[i,j] 
      + (P * gp.quicksum(z[i,q,j] for q in range(m)))
    )

In [33]:
# Equação 3
for j in range(n):
  for l in range(n):
    for i in range(o-1):
      for k in range(o):
        for q in range(m):
          model.addConstr
          (
            y[i+1,j] 
            >= y[k,l]
            - (M * x[q,i,j,k,l])
            - (M * (1 - z[i,q,j]))
            - (M * (1 - z[k,q,l]))
          )

In [34]:
# Equação 4 / (4-8)
for j in range(n):
  for l in range(n):
    for i in range(o-1):
      for k in range(o):
        for q in range(m):
          model.addConstr(y[k,l] >= y[i+1,j] - (M * (1 - x[q,i,j,k,l])) - (M * (1 - z[i,q,j])) - (M * (1 - z[k,q,l])))

In [35]:
# Equação 5 / (4-4)
for j in range(n):
  model.addConstr(gp.quicksum(z[0,q,j] for q in range(m)) <= 1)

In [36]:
# Equação 6 / (4-10)
for j in range(n):
  for l in range(n):
    if (j != l):
      for i in range(o-1): 
        for k in range(o):
          if (k > 0):
            for q in range(m):
              model.addConstr(x[q,i,j,k,l] + gp.quicksum(x[s,k-1,l,i+1,j] for s in range(m)) <= 1)

In [55]:
# Equação 7 / (4-9)
for j in range(n):
  for l in range(n):
    if (j != l):
      for i in range(o): 
        for k in range(o):
          for q in range(m):
            model.addConstr(x[q,i,j,k,l] + x[q,k,l,i,j] >= 1 - (M * (1 - z[i,q,j])) - (M * (1 - z[k,q,l])))

In [38]:
# Equação 8 / (4-3)
for j in range(n):
  for i in range(o):
    if (i > 1):
      model.addConstr
      (
        gp.quicksum(z[i,q,j] for q in range(m))
        == 1
        - gp.quicksum(f[k,j] for k in range(i-1))
      )

In [39]:
# Equação 9 / (4-7)
for j in TRAIN_OUT:
  for i in range(o):
    model.addConstr(f[i,j] <= z[i,PSA[j],j])

In [40]:
# Equação 10 / (4-5)
for j in TRAIN_OUT:
  model.addConstr(gp.quicksum(f[i,j] for i in range(o)) == 1)

In [41]:
# Equação 11 / (4-6)
for j in TRAIN_IN:
  model.addConstr(gp.quicksum(f[i,j] for i in range(o)) == 0)

In [42]:
# Equação 12 / (4-12)
for j in range(n):
  model.addConstr(z[0,PCH[j],j] == 1)

In [43]:
# Equação 13 / (4-13)
for j in TRAIN_OUT:
  model.addConstr(y[0,j] == RELEASE[j])

In [44]:
# Equação 14 / (4-14)
for j in TRAIN_OUT:
  model.addConstr(y[0,j] >= RELEASE[j])

In [45]:
# Equação 15 / (4-15)
for j in TRAIN_IN:
  for l in TRAIN_IN:
    model.addConstr(y[0,j] == y[0,l])

In [46]:
# Equação 16 / (4-11)
for j in range(n):
  for i in range(o-1):
    for q in range(m):
      model.addConstr(z[i+1,q,j] <= gp.quicksum(AMV[s][q] * z[i,s,j] for s in range(m)))

In [47]:
# Equação 17 / (4-16)
for j in TRAIN_OUT:
  for l in TRAIN_OUT:
    for i in range(o):
      model.addConstr(c[j] >= y[i,l])

In [48]:
# Equação 18 / (4-17)
for j in TRAIN_IN:
  model.addConstr(c[j] >= y[o-1,j])

In [49]:
# Equação 19 / (4-18)
for j in range(n):
  for i in range(o-1):
    model.addConstr(y[i+1,j] >= c[j] - M * (1 - gp.quicksum(f[k,j] for k in range(i))))

In [50]:
# Equação 20 / (4-19)
for j in TRAIN_OUT:
  model.addConstr(c_max >= c[j])

## Otimização do Problema

In [51]:
# Execute model
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 278 rows, 5643 columns and 882 nonzeros
Model fingerprint: 0xc2b6a3ee
Variable types: 33 continuous, 5610 integer (5610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 10.0000000
Presolve removed 278 rows and 5643 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


## Resultados Encontrados

In [54]:
for var in model.getVars():
    if (var.X > 0):
        print(f"{var.VarName} = {var.X}")

y[0,1] = 10.0
z[0,1,1] = 1.0
z[0,2,0] = 1.0
z[1,0,1] = 1.0
z[1,1,0] = 1.0
z[1,1,1] = 1.0
z[1,2,0] = 1.0
z[1,2,1] = 1.0
z[1,3,0] = 1.0
z[1,4,1] = 1.0
z[2,0,0] = 1.0
z[2,0,1] = 1.0
z[2,1,0] = 1.0
z[2,1,1] = 1.0
z[2,2,0] = 1.0
z[2,2,1] = 1.0
z[2,3,0] = 1.0
z[2,3,1] = 1.0
z[2,4,0] = 1.0
z[2,4,1] = 1.0
z[2,5,1] = 1.0
z[3,0,0] = 1.0
z[3,0,1] = 1.0
z[3,1,0] = 1.0
z[3,1,1] = 1.0
z[3,2,0] = 1.0
z[3,2,1] = 1.0
z[3,3,0] = 1.0
z[3,3,1] = 1.0
z[3,4,0] = 1.0
z[3,4,1] = 1.0
z[3,5,0] = 1.0
z[3,5,1] = 1.0
z[4,0,0] = 1.0
z[4,0,1] = 1.0
z[4,1,0] = 1.0
z[4,1,1] = 1.0
z[4,2,0] = 1.0
z[4,2,1] = 1.0
z[4,3,0] = 1.0
z[4,3,1] = 1.0
z[4,4,0] = 1.0
z[4,4,1] = 1.0
z[4,5,0] = 1.0
z[4,5,1] = 1.0
z[5,0,0] = 1.0
z[5,0,1] = 1.0
z[5,1,0] = 1.0
z[5,1,1] = 1.0
z[5,2,0] = 1.0
z[5,2,1] = 1.0
z[5,3,0] = 1.0
z[5,3,1] = 1.0
z[5,4,0] = 1.0
z[5,4,1] = 1.0
z[5,5,0] = 1.0
z[5,5,1] = 1.0
z[6,0,0] = 1.0
z[6,0,1] = 1.0
z[6,1,0] = 1.0
z[6,1,1] = 1.0
z[6,2,0] = 1.0
z[6,2,1] = 1.0
z[6,3,0] = 1.0
z[6,3,1] = 1.0
z[6,4,0] = 1.0
z[6,4,1] = 